# Optimizing Query Performance in Snowflake using AI Tools

Steps to optimizing query performance:
1. Identify queries that run the longest
2. Optimize the longest running queries

## Taking advantage of Snowflake Cortex Large Language Model (LLM) Functions

https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions

Available functions:
- COMPLETE, CLASSIFY_TEXT, EXTRACT_ANSWER, PARSE_DOCUMENT, SENTIMENT, SUMMARIZE, TRANSLATE, EMBED_TEXT_768, etc.

### The COMPLETE function supports many models (different models have different costs):

- snowflake-arctic, claude-3-5-sonnet, gemma-7b, jamba-1.5-mini, jamba-1.5-large, llama3-8b, llama3.2-3b, mistral-large, mixtral-8x7b, reka-core, reka-flash, etc.

## Step 1: Identify queries that run the longest

Look into QUERY_HISTORY.

Ask the LLM to write the query and compare the output from different models.

**Warning:** always check the provided code for common sense, feasibility, and syntax errors

In [ ]:
SET PROMPT = 'Please write a Snowflake query that identifies the 5 longest running queries within the last 7 days';

SELECT SNOWFLAKE.CORTEX.COMPLETE('snowflake-arctic', $PROMPT);

### Output from snowflake-arctic
```sql
SELECT query_text, start_time, end_time, execution_time  
FROM table(information_schema.query_history_by_session(session_id => '<your_session_id>',                                                         start_time => dateadd(day, -7, current_timestamp())))  
ORDER BY execution_time DESC  
LIMIT 5; 
```  
Please replace `<your_session_id>` with the desired session ID.

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE('llama3.2-1b', $PROMPT);

### Output from llama3.2-1b
```sql
WITH    -- Get the current date and time   
current_date AS (SELECT MAX(CURRENT_TIMESTAMP) AS timestamp FROM your_table),      
-- Get the queries that ran within the last 7 days   
last_7_days AS (
SELECT query_id, query_name, query_start_time, query_end_time,       
  EXTRACT(DAY FROM query_start_time) AS query_start_day,       
  EXTRACT(DAY FROM query_end_time) AS query_end_day,       
  EXTRACT(HOUR FROM query_start_time) AS query_start_hour,       
  EXTRACT(HOUR FROM query_end_time) AS query_end_hour,       
  EXTRACT(MINUTE FROM query_start_time) AS query_start_minute,       
  EXTRACT(MINUTE FROM query_end_time) AS query_end_minute,       
  EXTRACT(SECOND FROM query_start_time) AS query_start_second,       
  EXTRACT(SECOND FROM query_end_time) AS query_end_second     
  FROM your_table     
  WHERE EXTRACT(DAY FROM query_start_time) BETWEEN CURRENT_DATE - INTERVAL '7 days' 
  AND CURRENT_DATE     
  GROUP BY query_id, query_name, query_start_time, query_end_time),      
-- Get the 5 longest running queries
longest_running_queries AS (     
  SELECT query_id, query_name, query_start_time, query_end_time, query_start_day, query_end_day, query_start_hour, query_end_hour, 
  query_start_minute, query_end_minute, query_start_second, query_end_second,       
ROW_NUMBER() OVER (PARTITION BY query_id ORDER BY query_start_time DESC) AS row_num     
FROM last_7_days     
WHERE row_num = 1)    
-- Select the 5 longest running queries 
SELECT query_id, query_name, query_start_time, query_end_time, query_start_day, query_end_day, query_start_hour, query_end_hour,
  query_start_minute, query_end_minute, query_start_second, query_end_second 
FROM longest_running_queries 
WHERE row_num = 1 
ORDER BY query_start_time DESC 
LIMIT 5; 
```

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE('reka-flash', $PROMPT);

### Output from reka-flash
```sql
WITH RecentQueries AS (
  SELECT QUERY_ID, START_TIME, END_TIME, ELAPSED_TIME_MS / 1000.0 AS ELAPSED_SECONDS   
  FROM SNOWFLAKE_AUDIT_LOG   
  WHERE START_TIME >= DATE_SUB(CURRENT_DATE, 7) 
)  
SELECT QUERY_ID,   ELAPSED_SECONDS,   START_TIME 
FROM RecentQueries 
ORDER BY ELAPSED_SECONDS DESC 
LIMIT 5; 
```

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE('claude-3-5-sonnet', $PROMPT);

### Output from claude-3-5-sonnet
Offers two versions
```sql
SELECT QUERY_ID, QUERY_TEXT, DATABASE_NAME, SCHEMA_NAME, USER_NAME, ROLE_NAME, WAREHOUSE_NAME, START_TIME, END_TIME, 
  TOTAL_ELAPSED_TIME/1000 as EXECUTION_TIME_SECONDS, EXECUTION_STATUS 
FROM SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY 
WHERE START_TIME >= DATEADD(days, -7, CURRENT_TIMESTAMP())     
AND EXECUTION_STATUS = 'SUCCESS' 
ORDER BY TOTAL_ELAPSED_TIME DESC 
LIMIT 5; 
``` 
```sql 
SELECT QUERY_ID, LEFT(QUERY_TEXT, 100) || '...' as TRUNCATED_QUERY_TEXT, DATABASE_NAME, SCHEMA_NAME, USER_NAME, WAREHOUSE_NAME, 
  START_TIME, END_TIME, ROUND(TOTAL_ELAPSED_TIME/1000/60, 2) as EXECUTION_TIME_MINUTES, 
  ROUND(BYTES_SCANNED/1024/1024/1024, 2) as GB_SCANNED, ROUND(BYTES_WRITTEN/1024/1024/1024, 2) as GB_WRITTEN, EXECUTION_STATUS 
FROM SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY 
WHERE START_TIME >= DATEADD(days, -7, CURRENT_TIMESTAMP())     
AND EXECUTION_STATUS = 'SUCCESS'     
AND TOTAL_ELAPSED_TIME > 0 
ORDER BY TOTAL_ELAPSED_TIME DESC 
LIMIT 5; 


## Execute the query to identify the longest running queries

Choose and modify a query provided by the LLM models

In [ ]:
SELECT query_id, query_text, start_time, end_time, total_elapsed_time
FROM table(information_schema.query_history())
WHERE start_time >= CURRENT_DATE - INTERVAL '7 days'
  AND end_time IS NOT NULL
ORDER BY total_elapsed_time DESC
LIMIT 5;

## Rewrite a query to optimize performance
Ask the LLM to rewrite the query and compare the output from different models.

Then execute the original query and the optimized query and compare performance.

**Remember:** when testing query performance, ensure the query doesn't read the result from the cahche by setting the USE_CACHED_RESULT parameter to False

In [ ]:
-- to test performance, ensure the query doesn't read the result from the cache
ALTER SESSION SET USE_CACHED_RESULT = FALSE;

In [ ]:
-- first example, rewrite one of the queries that select the longest running queries
SELECT SNOWFLAKE.CORTEX.COMPLETE('snowflake-arctic', 
  'Please rewrite the following Snowflake query as a single query without nesting the CTE: 

WITH query_stats AS (
  SELECT query_id, start_time, end_time, total_elapsed_time / 1000.0 AS total_elapsed_seconds
  FROM table(information_schema.query_history())
  WHERE start_time >= CURRENT_DATE - INTERVAL ''30 days''
)
SELECT query_id, total_elapsed_seconds AS query_duration_seconds
FROM query_stats
ORDER BY total_elapsed_seconds DESC
LIMIT 5
');

In [ ]:
-- output from the previous cell
SELECT query_id, total_elapsed_time / 1000.0 AS query_duration_seconds
FROM table(information_schema.query_history())
WHERE start_time >= CURRENT_DATE - INTERVAL '30 days'
ORDER BY total_elapsed_time DESC
LIMIT 5;

In [ ]:
-- second example, query from the Snowflake sample database
with cte1 as (
  select o_custkey, max(o_totalprice) as max_totalprice
  from SNOWFLAKE_SAMPLE_DATA.TPCH_SF100.ORDERS
  group by o_custkey
),
cte2 as (
  select c_custkey
  from SNOWFLAKE_SAMPLE_DATA.TPCH_SF100.CUSTOMER
  where c_mktsegment = 'BUILDING' or c_mktsegment = 'FURNITURE'
)
  select ORD.o_custkey, ORD.o_orderkey
  from SNOWFLAKE_SAMPLE_DATA.TPCH_SF100.ORDERS ORD
  inner join cte1 on cte1.o_custkey = ORD.o_custkey and cte1.max_totalprice = ORD.o_totalprice
  where ORD.o_custkey in (select c_custkey from cte2);

In [ ]:
-- second example, rewrite a query by the business
SELECT SNOWFLAKE.CORTEX.COMPLETE('snowflake-arctic', 
  'Please rewrite the following Snowflake query as a single query without nested CTEs using the qualify keyword: 
  
with cte1 as (
  select o_custkey, max(o_totalprice) as max_totalprice
  from SNOWFLAKE_SAMPLE_DATA.TPCH_SF100.ORDERS
  group by o_custkey
),
cte2 as (
  select c_custkey
  from SNOWFLAKE_SAMPLE_DATA.TPCH_SF100.CUSTOMER
  where c_mktsegment = ''BUILDING'' or c_mktsegment = ''FURNITURE''
)
  select ORD.o_custkey, ORD.o_orderkey
  from SNOWFLAKE_SAMPLE_DATA.TPCH_SF100.ORDERS ORD
  inner join cte1 on cte1.o_custkey = ORD.o_custkey and cte1.max_totalprice = ORD.o_totalprice
  where ORD.o_custkey in (select c_custkey from cte2)
');

In [ ]:
-- output from the LLM
SELECT o_custkey, o_orderkey
FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF100.ORDERS
WHERE o_custkey IN (
    SELECT c_custkey 
    FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF100.CUSTOMER 
    WHERE c_mktsegment IN ('BUILDING', 'FURNITURE')
)
QUALIFY o_totalprice = MAX(o_totalprice) OVER (PARTITION BY o_custkey);

## How much does it cost?

- To track credits used for AI Services including LLM Functions in your account, use the METERING_HISTORY view
- To view the credit and token consumption for each LLM function call, use the CORTEX_FUNCTIONS_USAGE_HISTORY view


In [ ]:
SELECT *
FROM SNOWFLAKE.ACCOUNT_USAGE.METERING_DAILY_HISTORY
WHERE SERVICE_TYPE = 'AI_SERVICES';


In [ ]:
SELECT * 
FROM SNOWFLAKE.ACCOUNT_USAGE.CORTEX_FUNCTIONS_USAGE_HISTORY 
ORDER BY START_TIME DESC;

**Remember:**
- Always
- Track costs, since models have different costs